In [1]:
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import os
import time
import SimpleITK as sitk
import datetime
import sys
import random
import pydicom

from addict import Dict
import numpy as np
from django.db import close_old_connections
from mtk_lib.sitk import transform_image2image

######### SETUP
from django import setup
sys.path.insert(0, '/usr/local/share/methinks/stroke_web')
os.environ['DJANGO_SETTINGS_MODULE'] = 'stroke_web.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
setup()
####################
from stroke.models import Diagnostic, Image, Case, Patient, Configuration, Institution, NeuralModel, User
import az_stge_client

In [2]:
stroke_db_path = '/data/stroke/database/'
disk_mask_path = '/export/JEREMYDISK/NCCT/data/atlas_int16/distmask_int16.nii.gz'
atlas_distmask= sitk.ReadImage(disk_mask_path)

In [3]:
overlays_path = '/export/NASDISK/cgranes/tmp/ad_tici3/ctrl'
images_path = '/export/NASDISK/cgranes/tmp/ad_tici3/images'

In [4]:
def create_empty_diagnostic_overlay(img):
    diagnostic = Diagnostic()
    diagnostic.image = img
    diagnostic.type = Diagnostic.OVERLAY
    diagnostic.data = {"aspects": {}, "version": "1.0", 
                       "segmentation": {'lesions': {"Chronic": {"ml": "0.00","code": 3,"color": "#06337a"},
                                                    "Subacute": {"ml": "0.00","code": 1,"color": "#295e6e"},
                                                    "Hemorrhage": {"ml": "0.00","code": 2,"color": "#dd3e1f"}
                                                   }
                                       }}

    diagnostic.date = datetime.datetime.now()
    if 'DL' in img.patient.uid:
        diagnostic.notes = 'DWI'
    else:
        diagnostic.notes = 'Control'
    diagnostic.save()
    return diagnostic

def hex2rgb(value):
    value = value.lstrip('#')
    return list(int(value[i:i+2], 16) for i in (0, 2 ,4))

def format_ml(ml):
    return '%.2f'%ml if ml<10 else '%.1f'%ml if ml<100 else '%.0f'%ml

def calculate_segmentation_data_for_ncct(diagnostic_data, diagnostic_model, brain_mask):
    max_lesion_code = max([details['code'] for _, details in diagnostic_model.data['segmentation']['lesions'].items()])
#     we need to add one as count start at 0 (background class)
    label_counts = np.bincount(diagnostic_data.flatten(), minlength= max_lesion_code + 1)
    voxel_ml = diagnostic_model.ml_per_voxel()
    seg_ml = list([c*diagnostic_model.ml_per_voxel() for c in label_counts])
    lesions = {}
    data = {}
    if brain_mask is not None:
        total_brain_volume = np.sum((brain_mask != 0).astype(np.int32)) * voxel_ml
        data['Brain'] = {'ml': np.around(total_brain_volume, decimals=4), 'code': -1}
        
    for lesion, details in diagnostic_model.data['segmentation']['lesions'].items():
        lesions[lesion] = {}
        for detail_key, detail_content in details.items():
            lesions[lesion][detail_key] = detail_content
        lesions[lesion]['ml'] = format_ml(seg_ml[details['code']])
        data[lesion] = {'ml': np.around(seg_ml[details['code']], decimals=4), 'code': details['code']}
    json_data = {'lesions': lesions, 'data': data}
    return json_data


def update_diagnostic_model(ncct, diagnostic, file_path, img, seg_data, registration):
    diagnostic.nii_path = file_path
    ## compute metrics:
    if registration is not None:
        dist_mask = sitk.GetArrayFromImage(transform_image2image(ncct, atlas_distmask,
                                                                 registration, direction='backward', order=0))
        relevant_volume_mask = (dist_mask > 0).astype(np.int16)
    else:
        relevant_volume_mask = None
    
    segmentation_data = calculate_segmentation_data_for_ncct(seg_data, diagnostic, relevant_volume_mask)
    diagnostic.data['segmentation'] = segmentation_data
    diagnostic.save()

In [5]:
def update_diag(diag, ncct, img, seg_img, orig_seg_path, registration):
    inference_folder = img.get_nii_path().replace('.nii.gz','')
    seg_path = os.path.join(inference_folder, '%d_O.nii.gz' % (diag.id) )
    
    
    print(seg_path)
    #### copy nifti to path in stroke (provisional)
    os.system(f"mkdir -p {os.path.dirname(seg_path)}")
    os.system(f"cp {orig_seg_path} {seg_path}")
    ## upload to azure
    az_stge_client.upload(seg_path)
    
    #'''
    seg_data = sitk.GetArrayFromImage(seg_img)
    update_diagnostic_model(ncct, diag, seg_path.replace(stroke_db_path,''), 
                            img, seg_data, registration)
    diag.save()#'''
    return diag

def create_diag(ncct, img, seg_img, orig_seg_path, registration):
    diag = create_empty_diagnostic_overlay(img)
    print('empty',diag.id)
    d = update_diag(diag, ncct, img, seg_img, orig_seg_path, registration)
    return d

In [6]:
for f in os.listdir(overlays_path):
    image_id = int(f.split('_')[1].replace('.nii.gz',''))
    print(f, image_id)
    image = Image.objects.get(id=image_id)
    #'''
    ncct = sitk.ReadImage(os.path.join(images_path, f))
    orig_seg_path = os.path.join(overlays_path, f)
    seg_img = sitk.ReadImage(orig_seg_path)
    
    d=create_diag(ncct, image, seg_img, orig_seg_path, registration=image.registration)
    print('created ',d.id)#'''

VH09372_51207.nii.gz 51207
empty 190977
/data/stroke/database/vh_treated/images/VH09372/51207_NCCT/190977_O.nii.gz
created  190977
VH11297_51215.nii.gz 51215
empty 190978
/data/stroke/database/vh_treated/images/VH11297/51215_NCCT/190978_O.nii.gz
created  190978
VH03870_51245.nii.gz 51245
empty 190979
/data/stroke/database/vh_treated/images/VH03870/51245_NCCT/190979_O.nii.gz
created  190979
VH39312_51249.nii.gz 51249
empty 190980
/data/stroke/database/vh_treated/images/VH39312/51249_NCCT/190980_O.nii.gz
created  190980
VH14780_51257.nii.gz 51257
empty 190981
/data/stroke/database/vh_treated/images/VH14780/51257_NCCT/190981_O.nii.gz
created  190981
VH57033_51283.nii.gz 51283
empty 190982
/data/stroke/database/vh_treated/images/VH57033/51283_NCCT/190982_O.nii.gz
created  190982
VH52974_51291.nii.gz 51291
empty 190983
/data/stroke/database/vh_treated/images/VH52974/51291_NCCT/190983_O.nii.gz
created  190983
VH30427_51337.nii.gz 51337
empty 190984
/data/stroke/database/vh_treated/images/VH3

created  191039
VH33188_53012.nii.gz 53012
empty 191040
/data/stroke/database/vh_treated/images/VH33188/53012_NCCT/191040_O.nii.gz
created  191040
VH48302_52352.nii.gz 52352
empty 191041
/data/stroke/database/vh_treated/images/VH48302/52352_NCCT/191041_O.nii.gz
created  191041
VH56568_53069.nii.gz 53069
empty 191042
/data/stroke/database/vh_treated/images/VH56568/53069_NCCT/191042_O.nii.gz
created  191042
VH96633_53100.nii.gz 53100
empty 191043
/data/stroke/database/vh_treated/images/VH96633/53100_NCCT/191043_O.nii.gz
created  191043
VH68526_53156.nii.gz 53156
empty 191044
/data/stroke/database/vh_treated/images/VH68526/53156_NCCT/191044_O.nii.gz
created  191044
VH41628_53171.nii.gz 53171
empty 191045
/data/stroke/database/vh_treated/images/VH41628/53171_NCCT/191045_O.nii.gz
created  191045
VH00921_53182.nii.gz 53182
empty 191046
/data/stroke/database/vh_treated/images/VH00921/53182_NCCT/191046_O.nii.gz
created  191046
VH76451_53185.nii.gz 53185
empty 191047
/data/stroke/database/vh_tr

created  191102
VH89984_55594.nii.gz 55594
empty 191103
/data/stroke/database/vh_treated/images/VH89984/55594_NCCT/191103_O.nii.gz
created  191103
VH33746_55606.nii.gz 55606
empty 191104
/data/stroke/database/vh_treated/images/VH33746/55606_NCCT/191104_O.nii.gz
created  191104
VH68541_55624.nii.gz 55624
empty 191105
/data/stroke/database/vh_treated/images/VH68541/55624_NCCT/191105_O.nii.gz
created  191105
VH26047_55658.nii.gz 55658
empty 191106
/data/stroke/database/vh_treated/images/VH26047/55658_NCCT/191106_O.nii.gz
created  191106
VH99490_55666.nii.gz 55666
empty 191107
/data/stroke/database/vh_treated/images/VH99490/55666_NCCT/191107_O.nii.gz
created  191107
VH39050_55789.nii.gz 55789
empty 191108
/data/stroke/database/vh_treated/images/VH39050/55789_NCCT/191108_O.nii.gz
created  191108
VH37404_55801.nii.gz 55801
empty 191109
/data/stroke/database/vh_treated/images/VH37404/55801_NCCT/191109_O.nii.gz
created  191109
VH34901_55827.nii.gz 55827
empty 191110
/data/stroke/database/vh_tr

created  191169
DL0809_5548.nii.gz 5548
empty 191170
/data/stroke/database/clinic/images/DL0809/5548_NCCT/191170_O.nii.gz
created  191170
DL1001_25988.nii.gz 25988
empty 191171
/data/stroke/database/clinic/images/DL1001/25988_NCCT/191171_O.nii.gz
created  191171
DL1005_25996.nii.gz 25996
empty 191172
/data/stroke/database/clinic/images/DL1005/25996_NCCT/191172_O.nii.gz
created  191172
DL1007_26000.nii.gz 26000
empty 191173
/data/stroke/database/clinic/images/DL1007/26000_NCCT/191173_O.nii.gz
created  191173
DL1009_26004.nii.gz 26004
empty 191174
/data/stroke/database/clinic/images/DL1009/26004_NCCT/191174_O.nii.gz
created  191174
DL1023_26024.nii.gz 26024
empty 191175
/data/stroke/database/clinic/images/DL1023/26024_NCCT/191175_O.nii.gz
created  191175
DL1025_26028.nii.gz 26028
empty 191176
/data/stroke/database/clinic/images/DL1025/26028_NCCT/191176_O.nii.gz
created  191176
DL1032_26038.nii.gz 26038
empty 191177
/data/stroke/database/clinic/images/DL1032/26038_NCCT/191177_O.nii.gz
cre